<a href="https://colab.research.google.com/github/jdesous1/MVP/blob/main/MVP_PUC_RIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MVP PUC RIO: Predição da Quantidade de Alunos a Serem Matriculados em uma Universidade com Base em Dados Históricos